# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096945


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<00:59,  2.12s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:48,  1.88s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:30,  1.33s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:20,  1.05it/s]

...

Rendering models:  70%|███████   | 21/30 [00:07<00:01,  4.53it/s]

Rendering models:  77%|███████▋  | 23/30 [00:08<00:01,  5.28it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  6.62it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  7.91it/s]

not-logged-in-3b0b929f4475fd276561      0.000137
Jnursssmith5263                         0.001099
not-logged-in-461bf2f260fc99e4866e      0.003183
Alexander00                             0.000261
jnarayanbvg                             0.000177
not-logged-in-c6917d9b19f41e6c2995      0.003280
salmi.a01                               0.001079
jaksonA                                 0.005335
TSM_Fatboy                              0.002883
xStormmyy                               0.001212
TRISTANM18                              0.000140
pangeli5                                0.001144
not-logged-in-9f30f411df9d28d2c541      0.002329
acapirala                               0.000256
Lavadude                              185.277725
helios1235                              0.000173
Tessaje04                               6.385110
equidad1                                0.000218
maxe83                                  0.002364
kgenere                                 0.000252
syt18               

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())